### 문자열 데이터 다루기-자연어 처리(감정분석)
order을 기준으로 문자열을 숫자로 바꾸는    알고리즘(임베딩)<br />
한국어는 별도처리 (순서에 상관없이 뜻이 통한다) <br />

In [47]:
from sklearn.datasets import load_files

### 데이터 불러오기 및 전처리

Windows에서 .tar.gz 파일을 추출하는 방법은 무엇입니까?
https://help.pressidium.com/en/articles/6979427-how-to-extract-and-unzip-tar-gz-files

In [23]:
reviews_train= load_files("data/aclImdb/train/")


In [48]:
text_train, y_train= reviews_train.data, reviews_train.target

In [49]:
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]


### 텍스트 임베딩 <br />
BOW <br />
장점:계산이 쉽다. <br/>
단점 : 데이터편향이 높다.

BOW, 불용어 참고자료 <br />
https://wikidocs.net/22650

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(text_train)
X_train =vect.transform(text_train)


In [51]:
feature_names = vect.get_feature_names_out()
feature_names[:10]

array(['00', '000', '0000', '0000000000000000000000000000000001',
       '0000000000001', '000000001', '000000003', '00000001', '000001745',
       '00001'], dtype=object)

### Baseline

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np
scores = cross_val_score(LogisticRegression(max_iter=1000), X_train, y_train, n_jobs=-1)
print(np.mean(scores))

0.6942266666666667


In [52]:
from sklearn.model_selection import GridSearchCV
param_grid = {"C" : [0.001,0.01,0.1,1,10]}
grid=GridSearchCV(LogisticRegression(max_iter=5000), param_grid, n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_score_)

KeyboardInterrupt: 

### 불용어 처리

In [45]:
vect = CountVectorizer(min_df=5, stop_words="english").fit(X_train)
X_train =vect.transform(text_train)
feature_names = vect.get_feature_names_out()

AttributeError: 'csr_matrix' object has no attribute 'lower'

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {"C" : [0.001,0.01,0.1,1,10]}
grid=GridSearchCV(LogisticRegression(max_iter=5000), param_grid, n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_score_)

### n-그램 -모르면 n=2를 쓰자
bow표현방식을 사용할 때 문맥을 고려하는 방법<br />
토큰 하나의 횟수만을 고려하지 않고 옆에 있는 두 세개의 토큰을 함께 고려하는 방식 <br/>
예) 네이버 연관 검색어 , 셜록홈즈하면 영국이 연관되어 뜬다

n그램 참고자료
https://wikidocs.net/21692

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression(max_iter=100)) #5개이하단어 무시, max_iter은 빠른 확인을 위해 100만 함. 나중에 5000으로 수정하기


In [41]:
param_grid= {"logisticRegression__C": [10,100],
             "tfidfVectorizer__ngram_range" :[(1,1),(1,2),(1,3)]}


In [46]:
grid=GridSearchCV(pipe, param_grid, n_jobs=-1)
grid.fit(text_train, y_train)
grid.best_score_

ValueError: Invalid parameter 'C' for estimator Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(min_df=5)),
                ('logisticregression', LogisticRegression())]). Valid parameters are: ['memory', 'steps', 'verbose'].